In [1]:
from pysat.formula import CNF
from pysat.solvers import *
import re
import pandas as pd
import os

CONFIG_DIR = "../linux-4.14.152"
DIMACS_FLA = 'examples/out.dimacs'


In [2]:
# playing a bit with the formula
fla = CNF('examples/out.dimacs')
# sanity check
with Solver(bootstrap_with=fla.clauses) as l:
    print(l.solve(assumptions=[]))
    print(l.solve(assumptions=[-73]))
    print(l.solve(assumptions=[-133]))
    print(l.solve(assumptions=[-75])) # X86 is a core feature
    print(l.solve(assumptions=[20568, 20569])) # semantics of _MODULE variables (cannot be y and m at the same time)

True
True
True
False
False


In [3]:
variables = {}
with open(DIMACS_FLA, 'r') as f:
    dimacs_lines = f.readlines()
    for line in dimacs_lines:
        if line.startswith("c"):
            m = re.search('(c) (\d+) (\w+)', line)
            var_id = int(m.group(2))
            var_name = m.group(3)  
            variables[var_id] = var_name
            # print(var_id, ":", var_name)


def search_kconfig_type(option_name, df_options):
    ktype = df_options.query("option == " + '"' + option_name + '"')['type']
    return ktype.values[0]

# we are thinking the DIMACS id (number k) corresponding to the "module" variable of an option
def search_kmodule(option_name):
    for k, v in variables.items():
        if v == option_name + "_MODULE":
            return k

In [4]:
# the goal is to check whether some options' values are *not* possible in the formula
# (or whether some values are forced)

# dictionary that associate options and values it can't take
impossible_values = dict()

def add_impossible_value(option, value):
    if option not in impossible_values:
        impossible_values[option] = {value}
    else:
        impossible_values[option].add(value)

def sanity_check_optionsvalues():
    fla = CNF(DIMACS_FLA)
    # we read once and for all
    df_options = pd.read_csv('examples/alloptions-x86.4.14.152.csv')
    # sanity check
    with Solver(bootstrap_with=fla.clauses) as l:
        print("at the beginning, fla is", l.solve(assumptions=[]))
        for k, v in variables.items():
            # TODO: if I will remember there is a true option called SOMETHINGXXX_MODULE
            # not in Linux options (dummy variables to emulate tristate)
            # not in Linux options (dummy variables to emulate choices)
            if (v.endswith("_MODULE") or "CHOICE_" in v):  
                continue # ignore
            kconfig_type = search_kconfig_type(v, df_options)

            if (kconfig_type == 'BOOL'):
                # we try true/false values                
                if not l.solve(assumptions=[k]):
                    print(k, "cannot take the 'y' value (dead feature)", v)
                    add_impossible_value(v, 'y')
                if not l.solve(assumptions=[-k]):
                    print(k, "cannot take the 'n' value (core feature)", v)
                    add_impossible_value(v, 'n')
            elif(kconfig_type == 'TRISTATE'):
                # we have to find the other "module" variable that emulates the "m" value 
                kmodule = search_kmodule(v)
                # we have to try y, n, and m
                # yes: k and !kmodule
                if not l.solve(assumptions=[k, -kmodule]):
                    print(k, "cannot take the 'y' value", v)
                    add_impossible_value(v, 'y')
                # no: !k and !kmodule
                if not l.solve(assumptions=[-k, -kmodule]):
                    print(k, "cannot take the 'n' value", v)
                    add_impossible_value(v, 'n')
                # module: !k and kmodule 
                if not l.solve(assumptions=[-k, kmodule]):
                    print(k, "cannot take the 'm' value", v)
                    add_impossible_value(v, 'm')
                # this case is not possible: k and kmodule
                # print("trying", v, k, "with three values")
            elif(kconfig_type == 'STRING'):
                continue # TODO?
            elif(kconfig_type == 'INT'):
                continue # TODO?
            else:
                continue # TODO?
            # print(k, v)
    
sanity_check_optionsvalues()

at the beginning, fla is True
20294 cannot take the 'y' value OMAP_SSI
20294 cannot take the 'm' value OMAP_SSI
8126 cannot take the 'y' value (dead feature) FB_MB862XX_LIME
18999 cannot take the 'y' value MTD_PXA2XX
18999 cannot take the 'm' value MTD_PXA2XX
838 cannot take the 'y' value (dead feature) PINCTRL_ARMADA_37XX
47989 cannot take the 'y' value (dead feature) ARCH_INLINE_WRITE_LOCK_BH
29027 cannot take the 'y' value MMC_SDHCI_BRCMSTB
29027 cannot take the 'm' value MMC_SDHCI_BRCMSTB
40909 cannot take the 'y' value (dead feature) IBM_EMAC_ZMII
10160 cannot take the 'n' value (core feature) HAVE_DMA_CONTIGUOUS
41590 cannot take the 'y' value (dead feature) RESET_OXNAS
32700 cannot take the 'y' value (dead feature) PAS_JOYSTICK
50394 cannot take the 'y' value (dead feature) B43_PHY_LCN
28908 cannot take the 'y' value SND_SOC_UX500_PLAT_MSP_I2S
28908 cannot take the 'm' value SND_SOC_UX500_PLAT_MSP_I2S
10031 cannot take the 'n' value (core feature) HAVE_RCU_TABLE_FREE
36080 canno

25457 cannot take the 'y' value (dead feature) FB_OMAP_LCDC_HWA742
73154 cannot take the 'y' value (dead feature) PINCTRL_SUN5I
37158 cannot take the 'y' value XILINX_SYSACE
37158 cannot take the 'm' value XILINX_SYSACE
5302 cannot take the 'n' value (core feature) PERF_EVENTS
27516 cannot take the 'y' value VIDEO_OMAP2_VOUT
27516 cannot take the 'm' value VIDEO_OMAP2_VOUT
858 cannot take the 'y' value (dead feature) PINCTRL_DIGICOLOR
22369 cannot take the 'y' value (dead feature) PINCTRL_S3C24XX
73392 cannot take the 'y' value DRM_OMAP_ENCODER_OPA362
73392 cannot take the 'm' value DRM_OMAP_ENCODER_OPA362
35873 cannot take the 'y' value SERIAL_AR933X
35873 cannot take the 'm' value SERIAL_AR933X
76031 cannot take the 'y' value (dead feature) ARM_S3C2410_CPUFREQ
68130 cannot take the 'y' value TEGRA_HOST1X
68130 cannot take the 'm' value TEGRA_HOST1X
35002 cannot take the 'y' value VIDEO_EXYNOS_FIMC_LITE
35002 cannot take the 'm' value VIDEO_EXYNOS_FIMC_LITE
16584 cannot take the 'y' v

37896 cannot take the 'y' value KEYBOARD_BFIN
37896 cannot take the 'm' value KEYBOARD_BFIN
67069 cannot take the 'y' value (dead feature) PCIE_QCOM
43175 cannot take the 'y' value (dead feature) SERIAL_SB1250_DUART_CONSOLE
19763 cannot take the 'y' value IBMVNIC
19763 cannot take the 'm' value IBMVNIC
25232 cannot take the 'y' value KEYBOARD_ATARI
25232 cannot take the 'm' value KEYBOARD_ATARI
38239 cannot take the 'y' value DASD_DIAG
38239 cannot take the 'm' value DASD_DIAG
10849 cannot take the 'y' value (dead feature) CLKSRC_NOMADIK_MTU
70234 cannot take the 'y' value PWM_SPEAR
70234 cannot take the 'm' value PWM_SPEAR
64965 cannot take the 'y' value I2C_XGENE_SLIMPRO
64965 cannot take the 'm' value I2C_XGENE_SLIMPRO
76277 cannot take the 'y' value (dead feature) FSL_BMAN_TEST_API
21601 cannot take the 'y' value (dead feature) SGI_SNSC
28121 cannot take the 'y' value ARM_RK3399_DMC_DEVFREQ
28121 cannot take the 'm' value ARM_RK3399_DMC_DEVFREQ
24472 cannot take the 'n' value (core

16486 cannot take the 'y' value (dead feature) PINCTRL_SH_PFC
53823 cannot take the 'y' value (dead feature) POWER_RESET_QNAP
65892 cannot take the 'y' value (dead feature) NET_VENDOR_APPLE
38800 cannot take the 'y' value I2C_ALTERA
38800 cannot take the 'm' value I2C_ALTERA
20971 cannot take the 'n' value (core feature) HAVE_PERF_EVENTS
2866 cannot take the 'y' value HI6220_MBOX
2866 cannot take the 'm' value HI6220_MBOX
209 cannot take the 'n' value (core feature) THREAD_INFO_IN_TASK
33499 cannot take the 'y' value SND_SOC_AU1XAC97C
33499 cannot take the 'm' value SND_SOC_AU1XAC97C
73174 cannot take the 'y' value IBMVETH
73174 cannot take the 'm' value IBMVETH
73110 cannot take the 'y' value (dead feature) LOCKDEP_SMALL
31639 cannot take the 'y' value TEGRA_ACONNECT
31639 cannot take the 'm' value TEGRA_ACONNECT
50333 cannot take the 'y' value (dead feature) SYS_SUPPORTS_SH_MTU2
48799 cannot take the 'y' value MTD_TSUNAMI
48799 cannot take the 'm' value MTD_TSUNAMI
46602 cannot take 

4907 cannot take the 'y' value PHYLINK
4907 cannot take the 'm' value PHYLINK
4439 cannot take the 'y' value (dead feature) PINCTRL_IMX6SL
64228 cannot take the 'y' value DRM_OMAP_CONNECTOR_ANALOG_TV
64228 cannot take the 'm' value DRM_OMAP_CONNECTOR_ANALOG_TV
8894 cannot take the 'y' value CRYPTO_DEV_PPC4XX
8894 cannot take the 'm' value CRYPTO_DEV_PPC4XX
2987 cannot take the 'y' value (dead feature) ARCH_USES_GETTIMEOFFSET
65551 cannot take the 'y' value SPIDER_NET
65551 cannot take the 'm' value SPIDER_NET
60922 cannot take the 'y' value FB_OMAP2_ENCODER_TFP410
60922 cannot take the 'm' value FB_OMAP2_ENCODER_TFP410
69555 cannot take the 'y' value HP_SDC
69555 cannot take the 'm' value HP_SDC
42682 cannot take the 'y' value (dead feature) MVEBU_GICP
18188 cannot take the 'y' value BFIN_OTP
18188 cannot take the 'm' value BFIN_OTP
50278 cannot take the 'y' value QCOM_BAM_DMA
50278 cannot take the 'm' value QCOM_BAM_DMA
59291 cannot take the 'y' value SND_SOC_UX500_PLAT_DMA
59291 cann

...

2676 cannot take the 'y' value (dead feature) SERIAL_PIC32_CONSOLE
60593 cannot take the 'y' value REGULATOR_QCOM_RPM
60593 cannot take the 'm' value REGULATOR_QCOM_RPM
75773 cannot take the 'y' value MOXART_DMA
75773 cannot take the 'm' value MOXART_DMA
35356 cannot take the 'y' value (dead feature) ARCH_OPTIONAL_KERNEL_RWX_DEFAULT
49445 cannot take the 'y' value PHY_QCOM_IPQ806X_SATA
49445 cannot take the 'm' value PHY_QCOM_IPQ806X_SATA
40276 cannot take the 'y' value SERIAL_MPC52xx
40276 cannot take the 'm' value SERIAL_MPC52xx
11657 cannot take the 'y' value SND_AOA_SOUNDBUS
11657 cannot take the 'm' value SND_AOA_SOUNDBUS
62308 cannot take the 'y' value SPARC_US2E_CPUFREQ
62308 cannot take the 'm' value SPARC_US2E_CPUFREQ
65507 cannot take the 'y' value FS_ENET_MDIO_FEC
65507 cannot take the 'm' value FS_ENET_MDIO_FEC
73072 cannot take the 'y' value MTD_NAND_CM_X270
73072 cannot take the 'm' value MTD_NAND_CM_X270
32345 cannot take the 'y' value PXA_FICP
32345 cannot take the 'm' 

23221 cannot take the 'y' value CRYPTO_DEV_FSL_CAAM_CRYPTO_API_DESC
23221 cannot take the 'm' value CRYPTO_DEV_FSL_CAAM_CRYPTO_API_DESC
73570 cannot take the 'y' value (dead feature) RESET_TEGRA_BPMP
23949 cannot take the 'y' value (dead feature) HAVE_S3C2410_WATCHDOG
34602 cannot take the 'y' value INDYDOG
34602 cannot take the 'm' value INDYDOG
39631 cannot take the 'y' value (dead feature) PINCTRL_PFC_R8A7792
57629 cannot take the 'y' value ARM_DT_BL_CPUFREQ
57629 cannot take the 'm' value ARM_DT_BL_CPUFREQ
53451 cannot take the 'y' value (dead feature) SOC_INTEGRATOR_CM
60205 cannot take the 'y' value DRM_OMAP_PANEL_LGPHILIPS_LB035Q02
60205 cannot take the 'm' value DRM_OMAP_PANEL_LGPHILIPS_LB035Q02
43883 cannot take the 'n' value (core feature) GENERIC_CLOCKEVENTS_MIN_ADJUST
36916 cannot take the 'y' value MMC_SDHCI_OF_HLWD
36916 cannot take the 'm' value MMC_SDHCI_OF_HLWD
35820 cannot take the 'y' value (dead feature) QCOM_PM
16192 cannot take the 'y' value (dead feature) NET_VEN

73134 cannot take the 'y' value CRYPTO_MD5_OCTEON
73134 cannot take the 'm' value CRYPTO_MD5_OCTEON
76283 cannot take the 'y' value SND_SGI_HAL2
76283 cannot take the 'm' value SND_SGI_HAL2
38654 cannot take the 'y' value ACPI_CPPC_CPUFREQ
38654 cannot take the 'm' value ACPI_CPPC_CPUFREQ
5667 cannot take the 'y' value (dead feature) DRM_EXYNOS_G2D
20073 cannot take the 'y' value PWM_JZ4740
20073 cannot take the 'm' value PWM_JZ4740
56302 cannot take the 'n' value (core feature) HAVE_OPTPROBES
10032 cannot take the 'n' value (core feature) HAVE_RCU_TABLE_INVALIDATE
72135 cannot take the 'y' value (dead feature) FB_WMT_GE_ROPS
45160 cannot take the 'y' value BFIN_CFPCMCIA
45160 cannot take the 'm' value BFIN_CFPCMCIA
9211 cannot take the 'm' value OF_PCI_IRQ
69679 cannot take the 'y' value (dead feature) PCI_ECAM
21780 cannot take the 'y' value (dead feature) NET_VENDOR_TUNDRA
21621 cannot take the 'y' value PCMCIA_SOC_COMMON
21621 cannot take the 'm' value PCMCIA_SOC_COMMON
4978 cannot

40244 cannot take the 'y' value (dead feature) SERIAL_AMBA_PL010_CONSOLE
6338 cannot take the 'y' value DRM_STM_DSI
6338 cannot take the 'm' value DRM_STM_DSI
73123 cannot take the 'y' value PERCPU_TEST
331 cannot take the 'y' value SCSI_OAK1
331 cannot take the 'm' value SCSI_OAK1
29235 cannot take the 'y' value SATA_FSL
29235 cannot take the 'm' value SATA_FSL
55825 cannot take the 'y' value RESET_TI_SCI
55825 cannot take the 'm' value RESET_TI_SCI
40888 cannot take the 'y' value (dead feature) BCMA_NFLASH
65934 cannot take the 'y' value VIDEO_IMX_CSI
65934 cannot take the 'm' value VIDEO_IMX_CSI
42455 cannot take the 'y' value SERIAL_SIRFSOC
42455 cannot take the 'm' value SERIAL_SIRFSOC
65665 cannot take the 'y' value SERIO_AMBAKMI
65665 cannot take the 'm' value SERIO_AMBAKMI
5657 cannot take the 'y' value TN3270
5657 cannot take the 'm' value TN3270
61522 cannot take the 'y' value TEST_BPF
1792 cannot take the 'y' value TILE_NET
1792 cannot take the 'm' value TILE_NET
49104 canno

69302 cannot take the 'y' value (dead feature) PINCTRL_WM8650
22382 cannot take the 'y' value SND_POWERMAC
22382 cannot take the 'm' value SND_POWERMAC
59117 cannot take the 'y' value MTD_VMU
59117 cannot take the 'm' value MTD_VMU
57068 cannot take the 'y' value LEDS_FSG
57068 cannot take the 'm' value LEDS_FSG
29143 cannot take the 'y' value (dead feature) CLPS711X_IRQCHIP
27071 cannot take the 'y' value DMA_SUN4I
27071 cannot take the 'm' value DMA_SUN4I
30206 cannot take the 'y' value TEST_USER_COPY
3496 cannot take the 'y' value (dead feature) GPIO_MPC5200
47196 cannot take the 'y' value (dead feature) USB_TUSB_OMAP_DMA
43064 cannot take the 'y' value TI_SCI_CLK
43064 cannot take the 'm' value TI_SCI_CLK
36855 cannot take the 'y' value (dead feature) MFD_T7L66XB
70793 cannot take the 'y' value (dead feature) SERIAL_EARLYCON_ARM_SEMIHOST


In [5]:
# demonstration of regular expression 
m = re.search('(c) (\d+) (\w+)', 'c 2130 TABLET_USB_PEGASUS')
print(m.group(2), m.group(3))

2130 TABLET_USB_PEGASUS


In [6]:
# demonstration: how to retrieve the DIMACS id of a variable name 
for k, v in variables.items():
    if v == "X86_64":
        print(k, v)

73 X86_64


In [7]:
# demonstration: how to retrieve the DIMACS id of a variable "module" name out of an option name
search_kmodule('TOSHIBA_WMI')

20756

In [48]:
def get_dict(filename):
    res = dict()
    with open(CONFIG, 'r') as f:
        config_lines = f.readlines()
        for line in config_lines:
            if not line.startswith("#") and line != '\n':
                m = re.search('CONFIG_(\w+)=([\w"-/]+)', line)
                var_name = m.group(1)
                var_val = m.group(2)
                res[var_name] = var_val
    return res

CONFIG = "../linux-4.14.152/.config"
config = get_dict(CONFIG)

In [13]:
config

{'64BIT': 'y',
 'X86_64': 'y',
 'X86': 'y',
 'INSTRUCTION_DECODER': 'y',
 'OUTPUT_FORMAT': '"elf64',
 'ARCH_DEFCONFIG': '"arch/x86/configs/x86_64_defconfig"',
 'LOCKDEP_SUPPORT': 'y',
 'STACKTRACE_SUPPORT': 'y',
 'MMU': 'y',
 'ARCH_MMAP_RND_BITS_MIN': '28',
 'ARCH_MMAP_RND_BITS_MAX': '32',
 'ARCH_MMAP_RND_COMPAT_BITS_MIN': '8',
 'ARCH_MMAP_RND_COMPAT_BITS_MAX': '16',
 'NEED_DMA_MAP_STATE': 'y',
 'NEED_SG_DMA_LENGTH': 'y',
 'GENERIC_BUG': 'y',
 'GENERIC_BUG_RELATIVE_POINTERS': 'y',
 'GENERIC_HWEIGHT': 'y',
 'RWSEM_XCHGADD_ALGORITHM': 'y',
 'GENERIC_CALIBRATE_DELAY': 'y',
 'ARCH_HAS_CPU_RELAX': 'y',
 'ARCH_HAS_CACHE_LINE_SIZE': 'y',
 'HAVE_SETUP_PER_CPU_AREA': 'y',
 'NEED_PER_CPU_EMBED_FIRST_CHUNK': 'y',
 'NEED_PER_CPU_PAGE_FIRST_CHUNK': 'y',
 'ARCH_HIBERNATION_POSSIBLE': 'y',
 'ARCH_SUSPEND_POSSIBLE': 'y',
 'ARCH_WANT_HUGE_PMD_SHARE': 'y',
 'ARCH_WANT_GENERAL_HUGETLB': 'y',
 'ZONE_DMA32': 'y',
 'AUDIT_ARCH': 'y',
 'ARCH_SUPPORTS_OPTIMIZED_INLINING': 'y',
 'ARCH_SUPPORTS_DEBUG_PAGEALLOC'

In [15]:
impossible_values

{'OMAP_SSI': {'m', 'y'},
 'FB_MB862XX_LIME': {'y'},
 'MTD_PXA2XX': {'m', 'y'},
 'PINCTRL_ARMADA_37XX': {'y'},
 'ARCH_INLINE_WRITE_LOCK_BH': {'y'},
 'MMC_SDHCI_BRCMSTB': {'m', 'y'},
 'IBM_EMAC_ZMII': {'y'},
 'HAVE_DMA_CONTIGUOUS': {'n'},
 'RESET_OXNAS': {'y'},
 'PAS_JOYSTICK': {'y'},
 'B43_PHY_LCN': {'y'},
 'SND_SOC_UX500_PLAT_MSP_I2S': {'m', 'y'},
 'HAVE_RCU_TABLE_FREE': {'n'},
 'PCI_HOST_COMMON': {'y'},
 'LUSTRE_TRANSLATE_ERRNOS': {'y'},
 'LEDS_S3C24XX': {'m', 'y'},
 'INLINE_WRITE_TRYLOCK': {'y'},
 'PINCTRL_TEGRA20': {'y'},
 'POWER_RESET_ST': {'y'},
 'SERIAL_KS8695': {'y'},
 'REGULATOR_TI_ABB': {'m', 'y'},
 'PINCTRL_ARTPEC6': {'y'},
 'ARMADA_375_CLK': {'y'},
 'MTD_LANTIQ': {'m', 'y'},
 'SND_SOC_SAMSUNG_SIMTEC': {'m', 'y'},
 'SND_DAVINCI_SOC_GENERIC_EVM': {'m', 'y'},
 'ARMADA_CP110_SYSCON': {'y'},
 'MMC_S3C': {'m', 'y'},
 'EDAC_ALTERA_SDMMC': {'y'},
 'GENERIC_FIND_FIRST_BIT': {'n'},
 'ARCH_HAS_ELF_RANDOMIZE': {'n'},
 'MTD_NAND_SLC_LPC32XX': {'m', 'y'},
 'OMAP2_DSS_DPI': {'y'},
 'CRYPTO

In [ ]:
def check_inconsistency(config_dict):
    res = dict()
    for feature in config_dict:
        if feature in impossible_values:
            if config_dict[feature] in impossible_values[feature]:
                res[feature] = config_dict[feature]
    return res

def rapporteur(d):
    if d:
        print()
        for k, v in d.items():
            print(k, "was set to an impossible value", v)
    else:
        print("[OK]")

def checker(n):
    for i in range(n):
        a = os.system("make randconfig -C {}".format(CONFIG_DIR))
        print(a)
        config_dict = get_dict(CONFIG)
        print("RANDCONFIG test {} :".format(i), end="")
        rapporteur(config_dict)
        

In [ ]:
checker(1000)

256
RANDCONFIG test 0 :[OK]
256
RANDCONFIG test 1 :[OK]
256
RANDCONFIG test 2 :[OK]
256
RANDCONFIG test 3 :[OK]
256
RANDCONFIG test 4 :[OK]
256
RANDCONFIG test 5 :[OK]
2
RANDCONFIG test 6 :[OK]
256
RANDCONFIG test 7 :[OK]
256
RANDCONFIG test 8 :[OK]
256
RANDCONFIG test 9 :[OK]
256
RANDCONFIG test 10 :[OK]
256
RANDCONFIG test 11 :[OK]
256
RANDCONFIG test 12 :[OK]
256
RANDCONFIG test 13 :[OK]
256
RANDCONFIG test 14 :[OK]
256
RANDCONFIG test 15 :[OK]
256
RANDCONFIG test 16 :[OK]
256
RANDCONFIG test 17 :[OK]
256
RANDCONFIG test 18 :[OK]
256
RANDCONFIG test 19 :[OK]
256
RANDCONFIG test 20 :[OK]
256
RANDCONFIG test 21 :[OK]
256
RANDCONFIG test 22 :[OK]
256
RANDCONFIG test 23 :[OK]
256
RANDCONFIG test 24 :[OK]
256
RANDCONFIG test 25 :[OK]
256
RANDCONFIG test 26 :[OK]
256
RANDCONFIG test 27 :[OK]
256
RANDCONFIG test 28 :[OK]
256
RANDCONFIG test 29 :[OK]
256
RANDCONFIG test 30 :[OK]
256
RANDCONFIG test 31 :[OK]
256
RANDCONFIG test 32 :[OK]
256
RANDCONFIG test 33 :[OK]
256
RANDCONFIG test 34 :[O